# Tahu lah apa ini?

In [ ]:
# Path to google drive folder
# %cd /content/drive/MyDrive/Colab Notebooks/Disertasi-Ahmad-Rofiqul/002. Laporan-eksperimen
# %cd /content/drive/MyDrive/Colab Notebooks/Disertasi-Ahmad-Rofiqul/002.Experiment_SWI_16052024/

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest, VotingClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from pyod.models.hbos import HBOS
from sklearn.cluster import DBSCAN
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score, roc_auc_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, OutlierMixin
from scipy.spatial.distance import mahalanobis

In [42]:
# Load the dataset
# file_path = '01.Dataset-Rofiqul/agriculture_dataset.csv'
# file_path = '01.Dataset-Rofiqul/Ujicoba3-29052024/agriculture_dataset.csv'
# file_path = 'dataset/agriculture_dataset.csv'
file_path = 'dataset/09_05_1_AgricultureOutliersResult_30052024.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset for review
# data.head(),data.info(),data.describe()
data

,Date,Year,Experiment,DataUse,Replication,Month,Vegetation,VegType,N2O,N_rate,...,SOM,if_outlier,lof_outlier,ocsvm_outlier,elliptic_outlier,hbos_outlier,dbscan_outlier,mahalanobis_distance,mahalanobis_outlier,majority_voting
0,2/9/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.896742,170,...,1.174072,1,1,1,1,0,0,0.125380,0,1
1,2/10/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,2.190218,170,...,1.174072,1,1,1,1,0,0,0.185283,0,1
2,2/18/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.542594,170,...,1.174072,1,1,1,1,0,0,0.137812,0,1
3,2/19/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.342870,170,...,1.174072,1,1,1,1,0,0,0.144823,0,1
4,3/16/12,2012,BCSE_KBS,Building,R1,March,Corn,Annual,2.947778,170,...,1.174072,1,1,1,1,0,0,0.158691,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,10/16/2013,2013,MCSE-T2,Testing,R4,October,TRIAE,Annual,1.640000,91,...,1.806000,1,1,1,1,0,0,0.204597,0,1
2242,11/15/2013,2013,MCSE-T2,Testing,R1,November,TRIAE,Annual,0.990000,91,...,1.496400,1,1,1,1,0,0,0.227413,0,1
2243,11/15/2013,2013,MCSE-T2,Testing,R2,November,TRIAE,Annual,-0.200000,91,...,1.186800,1,1,1,1,0,0,0.269185,0,1
2244,11/15/2013,2013,MCSE-T2,Testing,R3,November,TRIAE,Annual,0.300000,91,...,1.823200,1,1,1,1,0,0,0.251634,0,1


In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2246 entries, 0 to 2245
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  2246 non-null   object 
 1   Year                  2246 non-null   int64  
 2   Experiment            2246 non-null   object 
 3   DataUse               2246 non-null   object 
 4   Replication           2246 non-null   object 
 5   Month                 2246 non-null   object 
 6   Vegetation            2246 non-null   object 
 7   VegType               2246 non-null   object 
 8   N2O                   2246 non-null   float64
 9   N_rate                2246 non-null   int64  
 10  PP2                   2246 non-null   float64
 11  PP7                   2246 non-null   float64
 12  AirT                  2246 non-null   float64
 13  DAF_TD                2246 non-null   int64  
 14  DAF_SD                2246 non-null   int64  
 15  WFPS25cm             

In [44]:
# Convert outlier labels to binary (1 for outliers, 0 for inliers) for evaluation metrics
y_true = (data['majority_voting'] == -1).astype(int)

# Voting-based with averaging (soft voting)
data['average_voting'] = data[[f'{name}_outlier' for name, _ in models]].mean(axis=1).apply(lambda x: -1 if x < 0 else 1)


In [45]:
data

,Date,Year,Experiment,DataUse,Replication,Month,Vegetation,VegType,N2O,N_rate,...,if_outlier,lof_outlier,ocsvm_outlier,elliptic_outlier,hbos_outlier,dbscan_outlier,mahalanobis_distance,mahalanobis_outlier,majority_voting,average_voting
0,2/9/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.896742,170,...,1,1,1,1,0,0,0.125380,0,1,1
1,2/10/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,2.190218,170,...,1,1,1,1,0,0,0.185283,0,1,1
2,2/18/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.542594,170,...,1,1,1,1,0,0,0.137812,0,1,1
3,2/19/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.342870,170,...,1,1,1,1,0,0,0.144823,0,1,1
4,3/16/12,2012,BCSE_KBS,Building,R1,March,Corn,Annual,2.947778,170,...,1,1,1,1,0,0,0.158691,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,10/16/2013,2013,MCSE-T2,Testing,R4,October,TRIAE,Annual,1.640000,91,...,1,1,1,1,0,0,0.204597,0,1,1
2242,11/15/2013,2013,MCSE-T2,Testing,R1,November,TRIAE,Annual,0.990000,91,...,1,1,1,1,0,0,0.227413,0,1,1
2243,11/15/2013,2013,MCSE-T2,Testing,R2,November,TRIAE,Annual,-0.200000,91,...,1,1,1,1,0,0,0.269185,0,1,1
2244,11/15/2013,2013,MCSE-T2,Testing,R3,November,TRIAE,Annual,0.300000,91,...,1,1,1,1,0,0,0.251634,0,1,1


In [46]:
# Weighted voting
weights = {
    'if': 1,
    'lof': 1,
    'ocsvm': 1,
    'elliptic': 1,
    'hbos': 1,
    'dbscan': 1,
    'mahalanobis': 1
}
data['weighted_voting'] = data[[f'{name}_outlier' for name, _ in models]].apply(
    lambda row: np.average(row, weights=[weights[name] for name, _ in models]), axis=1).apply(lambda x: -1 if x < 0 else 1)


In [47]:
data

,Date,Year,Experiment,DataUse,Replication,Month,Vegetation,VegType,N2O,N_rate,...,lof_outlier,ocsvm_outlier,elliptic_outlier,hbos_outlier,dbscan_outlier,mahalanobis_distance,mahalanobis_outlier,majority_voting,average_voting,weighted_voting
0,2/9/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.896742,170,...,1,1,1,0,0,0.125380,0,1,1,1
1,2/10/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,2.190218,170,...,1,1,1,0,0,0.185283,0,1,1,1
2,2/18/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.542594,170,...,1,1,1,0,0,0.137812,0,1,1,1
3,2/19/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.342870,170,...,1,1,1,0,0,0.144823,0,1,1,1
4,3/16/12,2012,BCSE_KBS,Building,R1,March,Corn,Annual,2.947778,170,...,1,1,1,0,0,0.158691,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,10/16/2013,2013,MCSE-T2,Testing,R4,October,TRIAE,Annual,1.640000,91,...,1,1,1,0,0,0.204597,0,1,1,1
2242,11/15/2013,2013,MCSE-T2,Testing,R1,November,TRIAE,Annual,0.990000,91,...,1,1,1,0,0,0.227413,0,1,1,1
2243,11/15/2013,2013,MCSE-T2,Testing,R2,November,TRIAE,Annual,-0.200000,91,...,1,1,1,0,0,0.269185,0,1,1,1
2244,11/15/2013,2013,MCSE-T2,Testing,R3,November,TRIAE,Annual,0.300000,91,...,1,1,1,0,0,0.251634,0,1,1,1


In [48]:
# Save the updated data with outlier detection results
data.to_csv('dataset/09_05_2_1_AgricultureOutliersVotingBasedResult.csv', index=False)

In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2246 entries, 0 to 2245
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  2246 non-null   object 
 1   Year                  2246 non-null   int64  
 2   Experiment            2246 non-null   object 
 3   DataUse               2246 non-null   object 
 4   Replication           2246 non-null   object 
 5   Month                 2246 non-null   object 
 6   Vegetation            2246 non-null   object 
 7   VegType               2246 non-null   object 
 8   N2O                   2246 non-null   float64
 9   N_rate                2246 non-null   int64  
 10  PP2                   2246 non-null   float64
 11  PP7                   2246 non-null   float64
 12  AirT                  2246 non-null   float64
 13  DAF_TD                2246 non-null   int64  
 14  DAF_SD                2246 non-null   int64  
 15  WFPS25cm             